In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
!pip install numpy==1.15.4

In [3]:
# nakon što stavimo datoteke na Git uvijek čitati u ovomo obliku! (štedi vrijeme)
import time
start = time.time()
df = pd.read_hdf('./data/raw/training_dataset.h5', 'df')
end = time.time()
print(end - start)

# nakon što napravimo promjene kako treba, exportat ću ih u novu datoteku pomoću naredbe
# df.to_hdf('training_dataset2.h5', key='df')

4.433220386505127


In [4]:
# kasnije razmisliti o postavljanju na prosjek ovisno o vrsti klijenta? 
df['STAROST'].values[df['STAROST'].values >= 900] = -1
df['STAROST'].values[df['STAROST'].values < 0] = -1

In [5]:
df['PRIJEVREMENI_RASKID'] = np.where(df['DATUM_ZATVARANJA']>=(df['PLANIRANI_DATUM_ZATVARANJA']- np.timedelta64(10, 'D')), 'N', df['PRIJEVREMENI_RASKID'])

In [6]:
#brisanje outlier-a i ugovorenih iznosa = 0.0
df = df[(df['UGOVORENI_IZNOS'] < 2e5) & (df['UGOVORENI_IZNOS'] > 0.0)]

In [7]:
### BRISANJE PARTIJA S VIŠE KLIJENATA
start = time.time()
gb = df.groupby('OZNAKA_PARTIJE')['KLIJENT_ID'].nunique() # baci broj jedinstvenih klijenata za pojedinu partiju
# party_labels = df2['OZNAKA_PARTIJE'].unique().tolist() 
mask = gb[df['OZNAKA_PARTIJE']] == 1 # gb[df2['OZNAKA_PARTIJE']] vrati za svaku oznaku partije (svaki redak tablice) njenu vrijednost i usporedi je s 1
df = df[ mask.values ] # mask je neki objekt koji moram ovako koristiti, rezultat je skup gdje za svaku partiju ima jedan klijent
end = time.time()
print(end - start)

31.208325624465942


In [8]:
df = df[df['PLANIRANI_DATUM_ZATVARANJA'] > df['DATUM_OTVARANJA']] 

In [9]:
df.shape

(3830210, 17)

In [10]:
## POPUNJAVANJE MISSING VALUES UNAZAD (po grupama)

start = time.time()
# Create a groupby object
grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])
end = time.time()
print( end - start)

# Write a function that imputes median
def impute_backwards(series):
    return series.fillna(method = 'bfill')

start = time.time()
# Impute age and assign to titanic['age']
df['NOVI_DATUM_ZATVARANJA']  = grouped['DATUM_ZATVARANJA'].transform(impute_backwards)
end = time.time()
print(end - start)

start = time.time()
# Impute age and assign to titanic['age']
df['NOVA_VISINA_KAMATE']  = grouped['VISINA_KAMATE'].transform(impute_backwards)
end = time.time()
print(end - start)

0.0004956722259521484


KeyboardInterrupt: 

In [ ]:
## POPUNJAVANJE MISSING VALUES UNAZAD (po grupama)

start = time.time()
# Create a groupby object
grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])
end = time.time()
print( end - start)

# Write a function that imputes median
def impute_forwards(series):
    return series.fillna(method = 'ffill')

start = time.time()
# Impute age and assign to titanic['age']
df['NOVI_DATUM_ZATVARANJA2']  = grouped['NOVI_DATUM_ZATVARANJA'].transform(impute_forwards)
end = time.time()
print(end - start)

start = time.time()
# Impute age and assign to titanic['age']
df['NOVA_VISINA_KAMATE2']  = grouped['NOVA_VISINA_KAMATE'].transform(impute_forwards)
end = time.time()
print(end - start)

In [ ]:
missing_values = df.isnull().sum()
print(missing_values)

## pogledati jesu li uopće jedinstvene vrijednosti visine kamate u grupi, mozda se mijenjalo?

In [ ]:
df.shape

In [ ]:
n_unique = grouped['VISINA_KAMATE'].nunique()
print(n_unique)

In [ ]:
def impute_mean(series):
    return series.fillna(series.mean())


grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])

start = time.time()
df['AVG_VISINA_KAMATE']  = grouped['VISINA_KAMATE'].transform(impute_mean)
end = time.time()
print(end - start)

In [ ]:
print(df.isnull().sum())

In [ ]:
# Write a function that imputes median
def impute_backwards(series):
    return series.fillna(method = 'bfill')

prijevremeni_raskidi = df[df['PRIJEVREMENI_RASKID'] == 'Y']
group_raskidi = prijevremeni_raskidi.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])

start = time.time()
# Impute age and assign to titanic['age']
prijevremeni_raskidi['NOVI_DATUM_ZATVARANJA']  = group_raskidi['DATUM_ZATVARANJA'].transform(impute_backwards)
end = time.time()
print(end - start)

In [ ]:
# neka tu stoji ako zatreba
# sortiranje, potrebno za filanje missing values
# df.sort_values(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'DATUM_IZVJESTAVANJA', 'STANJE_NA_KRAJU_KVARTALA'], ascending=[True, True, True, True, False], inplace = True)

In [2]:
# export datoteke
df.to_hdf('./data/raw/nakon_procesiranja.h5', key='df')

NameError: name 'df' is not defined

In [ ]:
#od tu pokrecem na dalje(i import prvo)

In [2]:
df = pd.read_hdf('./data/raw/nakon_procesiranja.h5', 'df')

In [3]:
df.shape

(3830210, 22)

In [4]:
df.columns

Index(['DATUM_IZVJESTAVANJA', 'KLIJENT_ID', 'OZNAKA_PARTIJE',
       'DATUM_OTVARANJA', 'PLANIRANI_DATUM_ZATVARANJA', 'DATUM_ZATVARANJA',
       'UGOVORENI_IZNOS', 'STANJE_NA_KRAJU_PRETH_KVARTALA',
       'STANJE_NA_KRAJU_KVARTALA', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'VISINA_KAMATE', 'TIP_KAMATE', 'STAROST',
       'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA', 'NOVA_VISINA_KAMATE',
       'NOVI_DATUM_ZATVARANJA2', 'NOVA_VISINA_KAMATE2', 'AVG_VISINA_KAMATE'],
      dtype='object')

In [5]:
df_copy = df.copy()
del_col_list = ['DATUM_IZVJESTAVANJA','STANJE_NA_KRAJU_PRETH_KVARTALA', 'STANJE_NA_KRAJU_KVARTALA', 'DATUM_ZATVARANJA', 'NOVI_DATUM_ZATVARANJA', 'VISINA_KAMATE', 'NOVA_VISINA_KAMATE', 'NOVA_VISINA_KAMATE2']
df_copy.drop(del_col_list, axis = 1, inplace = True)
df_copy.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA',
       'PLANIRANI_DATUM_ZATVARANJA', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA2',
       'AVG_VISINA_KAMATE'],
      dtype='object')

In [6]:
print(df_copy.isnull().sum())

KLIJENT_ID                          0
OZNAKA_PARTIJE                      0
DATUM_OTVARANJA                     0
PLANIRANI_DATUM_ZATVARANJA          0
UGOVORENI_IZNOS                     0
VALUTA                              0
VRSTA_KLIJENTA                      0
PROIZVOD                            0
VRSTA_PROIZVODA                     0
TIP_KAMATE                          0
STAROST                             0
PRIJEVREMENI_RASKID                 0
NOVI_DATUM_ZATVARANJA2        1086755
AVG_VISINA_KAMATE                1086
dtype: int64


In [7]:
raskidi = df_copy[df_copy['PRIJEVREMENI_RASKID'] == 'Y']

In [8]:
print(raskidi.isnull().sum())

KLIJENT_ID                      0
OZNAKA_PARTIJE                  0
DATUM_OTVARANJA                 0
PLANIRANI_DATUM_ZATVARANJA      0
UGOVORENI_IZNOS                 0
VALUTA                          0
VRSTA_KLIJENTA                  0
PROIZVOD                        0
VRSTA_PROIZVODA                 0
TIP_KAMATE                      0
STAROST                         0
PRIJEVREMENI_RASKID             0
NOVI_DATUM_ZATVARANJA2          0
AVG_VISINA_KAMATE             247
dtype: int64


In [9]:
raskidi.dropna(subset = ['AVG_VISINA_KAMATE'], inplace = True)
raskidi.shape

C:\Users\e_len\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(266454, 14)

In [10]:
df_copy.dropna(subset = ['AVG_VISINA_KAMATE'], inplace = True)

In [11]:
df_copy.shape

(3829124, 14)

In [12]:
df_copy['NOVI_DATUM_ZATVARANJA2'] = np.where(df_copy['NOVI_DATUM_ZATVARANJA2'].isnull(), df_copy['PLANIRANI_DATUM_ZATVARANJA'] , df_copy['NOVI_DATUM_ZATVARANJA2'])

In [13]:
df_copy.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA',
       'PLANIRANI_DATUM_ZATVARANJA', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA2',
       'AVG_VISINA_KAMATE'],
      dtype='object')

In [14]:
df_copy.drop_duplicates(subset = ['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'], inplace = True)

In [15]:
df_copy.shape

(673448, 14)

In [16]:
df_copy.head()

,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,NOVI_DATUM_ZATVARANJA2,AVG_VISINA_KAMATE
1,772139,5521579,2010-06-11,2017-06-30,46000.00,1,1410,FL0801,A,B,38,N,2017-02-16,9.70
2,481985,7443823,2013-01-23,2018-02-28,44679.50,2,1550,FD0100,L,A,37,N,2015-09-07,3.10
3,139014,6511639,2011-09-16,2016-09-30,29959.15,2,1410,FL0801,A,B,49,N,2014-06-16,7.95
4,952566,6143123,2011-05-23,2014-05-31,100000.00,1,1410,FL0801,A,B,35,N,2014-04-10,9.70
6,686960,7791434,2013-05-27,2015-05-31,4017.00,1,1410,FL1100,A,A,65,N,2014-10-16,11.95


In [17]:
# duljina kredita je dana u danima
df_copy['PLANIRANA_DULJINA_KREDITA'] = (df_copy['PLANIRANI_DATUM_ZATVARANJA']-df_copy['DATUM_OTVARANJA']).dt.days

In [18]:
del df_copy['PLANIRANI_DATUM_ZATVARANJA']

In [19]:
df_copy.head()

,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,NOVI_DATUM_ZATVARANJA2,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA
1,772139,5521579,2010-06-11,46000.00,1,1410,FL0801,A,B,38,N,2017-02-16,9.70,2576
2,481985,7443823,2013-01-23,44679.50,2,1550,FD0100,L,A,37,N,2015-09-07,3.10,1862
3,139014,6511639,2011-09-16,29959.15,2,1410,FL0801,A,B,49,N,2014-06-16,7.95,1841
4,952566,6143123,2011-05-23,100000.00,1,1410,FL0801,A,B,35,N,2014-04-10,9.70,1104
6,686960,7791434,2013-05-27,4017.00,1,1410,FL1100,A,A,65,N,2014-10-16,11.95,734


In [20]:
# sluzi za merganje s makroekonomskim indikatorima
df_copy['GODINA_OTVARANJA']=df_copy['DATUM_OTVARANJA'].dt.year
df_copy['GODINA_ZATVARANJA']=df_copy['NOVI_DATUM_ZATVARANJA2'].dt.year

Makroekonomski indikatori

In [21]:
# Čitam tablicu ekonomskih indikatora
econ = pd.read_excel('./data/external/ekonomski_indikatori.xlsx')

In [22]:
econ.dropna(thresh=int(econ.shape[1] * .9), axis = 0, inplace = True)
column_names = econ.columns
# Drop unwanted columns
econ.drop(column_names[0], inplace=True, axis=1)

In [23]:
econ.shape
econ.head()

,General information on Croatia,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
3,NaN,2000,2001,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,2009.000000,2010.000000,2011.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018
4,Area (square km),56594,56594,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594
5,Population (million) a,4.426,4.30549,4.305384,4.305725,4.310861,4.312487,4.313530,4.311967,4.309796,4.302847,4.289857,4.280000,4.268000,4.256000,4.238000,4.204000,4.174000,4.125000,4.089
6,"GDP (million HRK, current prices) b",180241,194271,211500.384280,232444.050150,250564.762188,269779.451979,294150.258439,322463.846521,347749.878891,331367.071951,329143.142619,333456.571836,330824.955608,331784.893258,331569.648134,339616.204419,351349.058619,365643.149141,381799
7,"GDP (million EUR, current prices)",23610.7,26003.3,28554.214875,30729.300540,33427.889423,36456.451152,40168.827520,43956.244732,48143.611980,45148.120983,45173.312209,44854.374703,44008.247014,43808.383238,43455.968513,44629.962125,46663.725118,49013.169950,51496.3


In [24]:
econ.rename(columns={'Unnamed: 2': '2000', 'Unnamed: 3': '2001', 
                   'Unnamed: 4': '2002', 'Unnamed: 5': '2003', 
                   'Unnamed: 6': '2004', 'Unnamed: 7': '2005',
                  'Unnamed: 8': '2006', 'Unnamed: 9': '2007', 
                  'Unnamed: 10': '2008', 'Unnamed: 11': '2009',
                  'Unnamed: 12': '2010', 'Unnamed: 13': '2011',
                  'Unnamed: 14': '2012', 'Unnamed: 15': '2013',
                  'Unnamed: 16': '2014', 'Unnamed: 17': '2015',
                  'Unnamed: 18': '2016', 'Unnamed: 19': '2017',
                  'Unnamed: 20': '2018'}, inplace=True)

In [25]:
econ.head()

,General information on Croatia,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
3,NaN,2000,2001,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,2009.000000,2010.000000,2011.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018
4,Area (square km),56594,56594,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594
5,Population (million) a,4.426,4.30549,4.305384,4.305725,4.310861,4.312487,4.313530,4.311967,4.309796,4.302847,4.289857,4.280000,4.268000,4.256000,4.238000,4.204000,4.174000,4.125000,4.089
6,"GDP (million HRK, current prices) b",180241,194271,211500.384280,232444.050150,250564.762188,269779.451979,294150.258439,322463.846521,347749.878891,331367.071951,329143.142619,333456.571836,330824.955608,331784.893258,331569.648134,339616.204419,351349.058619,365643.149141,381799
7,"GDP (million EUR, current prices)",23610.7,26003.3,28554.214875,30729.300540,33427.889423,36456.451152,40168.827520,43956.244732,48143.611980,45148.120983,45173.312209,44854.374703,44008.247014,43808.383238,43455.968513,44629.962125,46663.725118,49013.169950,51496.3


In [26]:
econ.drop([4], axis = 0, inplace = True)

In [27]:
econ.head()

,General information on Croatia,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
3,NaN,2000,2001,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,2009.000000,2010.000000,2011.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018
5,Population (million) a,4.426,4.30549,4.305384,4.305725,4.310861,4.312487,4.313530,4.311967,4.309796,4.302847,4.289857,4.280000,4.268000,4.256000,4.238000,4.204000,4.174000,4.125000,4.089
6,"GDP (million HRK, current prices) b",180241,194271,211500.384280,232444.050150,250564.762188,269779.451979,294150.258439,322463.846521,347749.878891,331367.071951,329143.142619,333456.571836,330824.955608,331784.893258,331569.648134,339616.204419,351349.058619,365643.149141,381799
7,"GDP (million EUR, current prices)",23610.7,26003.3,28554.214875,30729.300540,33427.889423,36456.451152,40168.827520,43956.244732,48143.611980,45148.120983,45173.312209,44854.374703,44008.247014,43808.383238,43455.968513,44629.962125,46663.725118,49013.169950,51496.3
8,GDP per capita (in EUR),5334.56,6039.56,6632.210942,7136.846998,7754.341748,8453.695316,9312.286577,10194.012322,11170.740327,10492.615931,10530.260615,10479.994090,10311.210641,10293.323129,10253.885916,10616.070915,11179.617901,11881.980594,12593.9


In [28]:
econ2 = econ.transpose(copy = True)
new_column_names = econ2.iloc[0,:]
econ2.columns = new_column_names

In [29]:
econ2.reset_index()

General information on Croatia,index,nan,Population (million) a,"GDP (million HRK, current prices) b","GDP (million EUR, current prices)",GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, constant prices)",Average CPI year-on-year inflation rate,Current account balance (million EUR) c,Current account balance (as of % GDP),...,"Gross international reserves (in terms of months of imports of goods and services, end of year)",Exchange rate on 31 December (HRK : 1 EUR),Exchange rate on 31 December (HRK : 1 USD),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+)/borrowing (-) (million HRK) e,Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e,General government debt (as % of GDP) e,"Unemployment rate (ILO, persons above 15 years of age) f","Employment rate (ILO, persons above 15 years of age) f"
0,General information on Croatia,NaN,Population (million) a,"GDP (million HRK, current prices) b","GDP (million EUR, current prices)",GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, const...",Average CPI year-on-year inflation rate,Current account balance (million EUR) c,Current account balance (as of % GDP),...,Gross international reserves (in terms of mont...,Exchange rate on 31 December (HRK : 1 EUR),Exchange rate on 31 December (HRK : 1 USD),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+...,Consolidated general government net lending (+...,General government debt (as % of GDP) e,"Unemployment rate (ILO, persons above 15 years...","Employment rate (ILO, persons above 15 years o..."
1,2000,2000,4.426,180241,23610.7,5334.56,3.7675,4.6,-516.727,-2.18852,...,4.85331,7.59833,8.15534,7.63385,8.28737,....,....,35.4986,16.05,42.6
2,2001,2001,4.30549,194271,26003.3,6039.56,3.45241,3.8,-758.857,-2.91831,...,5.81721,7.37003,8.35604,7.47101,8.33915,....,....,36.4755,15.8,41.8
3,2002,2002,4.30538,211500,28554.2,6632.21,5.25336,1.7,-2016.88,-7.06332,...,5.20306,7.44229,7.14574,7.40698,7.87249,-7365.19,-3.48235,36.6133,14.8,43.3
4,2003,2003,4.30572,232444,30729.3,7136.85,5.58119,1.8,-1811.73,-5.89579,...,5.52068,7.64691,6.11851,7.56425,6.70445,-10863.8,-4.67371,38.1037,14.25,43.1
5,2004,2004,4.31086,250565,33427.9,7754.34,3.91456,2.1,-1355.35,-4.05455,...,5.0642,7.67123,5.63688,7.49568,6.03122,-13038.5,-5.20365,40.2784,13.8,43.5
6,2005,2005,4.31249,269779,36456.5,8453.7,4.10956,3.3,-1892.06,-5.18991,...,5.37051,7.37563,6.23363,7.40005,5.94996,-10413.5,-3.86001,41.1652,12.7,43.3
7,2006,2006,4.31353,294150,40168.8,9312.29,4.86782,3.2,-2612.7,-6.5043,...,5.60497,7.34508,5.5784,7.32285,5.83917,-9971.92,-3.39008,38.652,11.2,43.6
8,2007,2007,4.31197,322464,43956.2,10194,5.28365,2.9,-3138.26,-7.13952,...,5.48831,7.32513,4.98546,7.33602,5.36599,-7880.62,-2.44388,37.2593,9.9,47.6
9,2008,2008,4.3098,347750,48143.6,11170.7,2.03646,6.1,-4226.91,-8.7798,...,4.88824,7.32442,5.1555,7.22318,4.93442,-9604.64,-2.76194,39.0003,8.5,48.6


In [30]:
#econ2.drop([0], axis = 0, inplace = True)
indikatori = econ2.iloc[1:, :]
indikatori['GODINE']=range(2000,2019)

C:\Users\e_len\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
indikatori.head(19)

General information on Croatia,nan,Population (million) a,"GDP (million HRK, current prices) b","GDP (million EUR, current prices)",GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, constant prices)",Average CPI year-on-year inflation rate,Current account balance (million EUR) c,Current account balance (as of % GDP),Exports of goods and services (as of % GDP),...,Exchange rate on 31 December (HRK : 1 EUR),Exchange rate on 31 December (HRK : 1 USD),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+)/borrowing (-) (million HRK) e,Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e,General government debt (as % of GDP) e,"Unemployment rate (ILO, persons above 15 years of age) f","Employment rate (ILO, persons above 15 years of age) f",GODINE
2000,2000,4.426,180241,23610.7,5334.56,3.7675,4.6,-516.727,-2.18852,35.215,...,7.59833,8.15534,7.63385,8.28737,....,....,35.4986,16.05,42.6,2000
2001,2001,4.30549,194271,26003.3,6039.56,3.45241,3.8,-758.857,-2.91831,37.2483,...,7.37003,8.35604,7.47101,8.33915,....,....,36.4755,15.8,41.8,2001
2002,2002,4.30538,211500,28554.2,6632.21,5.25336,1.7,-2016.88,-7.06332,36.24,...,7.44229,7.14574,7.40698,7.87249,-7365.19,-3.48235,36.6133,14.8,43.3,2002
2003,2003,4.30572,232444,30729.3,7136.85,5.58119,1.8,-1811.73,-5.89579,39.7021,...,7.64691,6.11851,7.56425,6.70445,-10863.8,-4.67371,38.1037,14.25,43.1,2003
2004,2004,4.31086,250565,33427.9,7754.34,3.91456,2.1,-1355.35,-4.05455,39.7148,...,7.67123,5.63688,7.49568,6.03122,-13038.5,-5.20365,40.2784,13.8,43.5,2004
2005,2005,4.31249,269779,36456.5,8453.7,4.10956,3.3,-1892.06,-5.18991,39.4753,...,7.37563,6.23363,7.40005,5.94996,-10413.5,-3.86001,41.1652,12.7,43.3,2005
2006,2006,4.31353,294150,40168.8,9312.29,4.86782,3.2,-2612.7,-6.5043,39.771,...,7.34508,5.5784,7.32285,5.83917,-9971.92,-3.39008,38.652,11.2,43.6,2006
2007,2007,4.31197,322464,43956.2,10194,5.28365,2.9,-3138.26,-7.13952,39.0262,...,7.32513,4.98546,7.33602,5.36599,-7880.62,-2.44388,37.2593,9.9,47.6,2007
2008,2008,4.3098,347750,48143.6,11170.7,2.03646,6.1,-4226.91,-8.7798,38.4784,...,7.32442,5.1555,7.22318,4.93442,-9604.64,-2.76194,39.0003,8.5,48.6,2008
2009,2009,4.30285,331367,45148.1,10492.6,-7.29143,2.4,-2299.25,-5.09269,34.5037,...,7.3062,5.0893,7.33955,5.28037,-19844,-5.98853,48.2978,9.2,48.2,2009


In [32]:
indikatori.columns

Index([                                                                                              nan,
                                                                                'Population (million) a',
                                                                   'GDP (million HRK, current prices) b',
                                                                    'GDP (million EUR, current prices) ',
                                                                             'GDP per capita (in EUR)  ',
                                            'GDP - year-on-year rate of growth (in %, constant prices) ',
                                                              'Average CPI year-on-year inflation rate ',
                                                               'Current account balance (million EUR) c',
                                                                'Current account balance (as of % GDP) ',
                                              

In [33]:
drop_col = [0,1,2, 3,7,11, 13, 14, 15, 16, 17, 18, 24]
indikatori.columns[drop_col].values
indikatori.drop(indikatori.columns[drop_col].values, axis = 1, inplace = True)

C:\Users\e_len\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [34]:
indikatori.head()

General information on Croatia,GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, constant prices)",Average CPI year-on-year inflation rate,Current account balance (as of % GDP),Exports of goods and services (as of % GDP),Imports of goods and services (as of % GDP),External debt (as of % GDP),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+)/borrowing (-) (million HRK) e,Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e,General government debt (as % of GDP) e,"Employment rate (ILO, persons above 15 years of age) f",GODINE
2000,5334.56,3.7675,4.6,-2.18852,35.215,39.6182,51.9406,7.63385,8.28737,....,....,35.4986,42.6,2000
2001,6039.56,3.45241,3.8,-2.91831,37.2483,42.3117,52.3366,7.47101,8.33915,....,....,36.4755,41.8,2001
2002,6632.21,5.25336,1.7,-7.06332,36.24,45.6459,53.036,7.40698,7.87249,-7365.19,-3.48235,36.6133,43.3,2002
2003,7136.85,5.58119,1.8,-5.89579,39.7021,46.3605,64.7067,7.56425,6.70445,-10863.8,-4.67371,38.1037,43.1,2003
2004,7754.34,3.91456,2.1,-4.05455,39.7148,45.6236,68.6055,7.49568,6.03122,-13038.5,-5.20365,40.2784,43.5,2004


In [35]:
df_copy.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'UGOVORENI_IZNOS',
       'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA2',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'GODINA_ZATVARANJA'],
      dtype='object')

In [36]:
indikatori.columns

Index(['GDP per capita (in EUR)  ',
       'GDP - year-on-year rate of growth (in %, constant prices) ',
       'Average CPI year-on-year inflation rate ',
       'Current account balance (as of % GDP) ',
       'Exports of goods and services (as of % GDP) ',
       'Imports of goods and services (as of % GDP) ',
       'External debt (as of % GDP) ', 'Average exchange rate (HRK : 1 EUR) ',
       'Average exchange rate (HRK : 1 USD) ',
       'Consolidated general government net lending (+)/borrowing (-) (million HRK) e',
       'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e',
       'General government debt (as % of GDP) e',
       'Employment rate (ILO, persons above 15 years of age) f', 'GODINE'],
      dtype='object', name='General information on Croatia')

In [37]:
import time
start = time.time()
df_merged = pd.merge(df_copy, indikatori, left_on=['GODINA_OTVARANJA'], right_on = ['GODINE'], how = 'left')
end = time.time()
print(end-start)
df_merged.rename(columns={'GDP per capita (in EUR)  ': 'ind1-poc', 'GDP - year-on-year rate of growth (in %, constant prices) ': 'ind2-poc', 
                   'Average CPI year-on-year inflation rate ': 'ind3-poc', 'Current account balance (as of % GDP) ': 'ind4-poc', 
                   'Exports of goods and services (as of % GDP) ': 'ind5-poc', 'Imports of goods and services (as of % GDP) ': 'ind6-poc',
                  'External debt (as of % GDP) ': 'ind7-poc', 'Average exchange rate (HRK : 1 EUR) ': 'ind8-poc', 'Average exchange rate (HRK : 1 USD) ': 'ind9-poc',
                  'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e' : 'ind10-poc', 'General government debt (as % of GDP) e': 'ind11-poc',
                  'Employment rate (ILO, persons above 15 years of age) f': 'ind12-poc'}, inplace=True)

0.8749444484710693


In [38]:
del df_merged['Consolidated general government net lending (+)/borrowing (-) (million HRK) e']
del df_merged['GODINE']

In [39]:
df_merged.head()

,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,...,ind3-poc,ind4-poc,ind5-poc,ind6-poc,ind7-poc,ind8-poc,ind9-poc,ind10-poc,ind11-poc,ind12-poc
0,772139,5521579,2010-06-11,46000.00,1,1410,FL0801,A,B,38,...,1.1,-1.0676,37.6476,37.982,103.841,7.28623,5.50002,-6.27526,57.3479,46.5
1,481985,7443823,2013-01-23,44679.50,2,1550,FD0100,L,A,37,...,2.2,0.945846,42.8399,42.4593,104.554,7.57355,5.70588,-5.2795,80.3993,42.075
2,139014,6511639,2011-09-16,29959.15,2,1410,FL0801,A,B,49,...,2.3,-0.698502,40.415,40.8333,103.44,7.4342,5.34351,-7.86303,63.8686,44.8
3,952566,6143123,2011-05-23,100000.00,1,1410,FL0801,A,B,35,...,2.3,-0.698502,40.415,40.8333,103.44,7.4342,5.34351,-7.86303,63.8686,44.8
4,686960,7791434,2013-05-27,4017.00,1,1410,FL1100,A,A,65,...,2.2,0.945846,42.8399,42.4593,104.554,7.57355,5.70588,-5.2795,80.3993,42.075


In [40]:
start = time.time()
df_merged2 = pd.merge(df_merged, indikatori, left_on=['GODINA_ZATVARANJA'], right_on = ['GODINE'], how = 'left')
end = time.time()
print(end-start)

df_merged2.rename(columns={'GDP per capita (in EUR)  ': 'ind1-zav', 'GDP - year-on-year rate of growth (in %, constant prices) ': 'ind2-zav', 
                   'Average CPI year-on-year inflation rate ': 'ind3-zav', 'Current account balance (as of % GDP) ': 'ind4-zav', 
                   'Exports of goods and services (as of % GDP) ': 'ind5-zav', 'Imports of goods and services (as of % GDP) ': 'ind6-zav',
                  'External debt (as of % GDP) ': 'ind7-zav', 'Average exchange rate (HRK : 1 EUR) ': 'ind8-zav', 'Average exchange rate (HRK : 1 USD) ': 'ind9-zav',
                  'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e' : 'ind10-zav', 'General government debt (as % of GDP) e': 'ind11-zav',
                  'Employment rate (ILO, persons above 15 years of age) f': 'ind12-zav'}, inplace=True)


1.5951166152954102


In [41]:
del df_merged2['Consolidated general government net lending (+)/borrowing (-) (million HRK) e']
del df_merged2['GODINE']
df_merged2.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'UGOVORENI_IZNOS',
       'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA2',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'GODINA_ZATVARANJA', 'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc',
       'ind5-poc', 'ind6-poc', 'ind7-poc', 'ind8-poc', 'ind9-poc', 'ind10-poc',
       'ind11-poc', 'ind12-poc', 'ind1-zav', 'ind2-zav', 'ind3-zav',
       'ind4-zav', 'ind5-zav', 'ind6-zav', 'ind7-zav', 'ind8-zav', 'ind9-zav',
       'ind10-zav', 'ind11-zav', 'ind12-zav'],
      dtype='object')

In [42]:
df_merged.shape

(673448, 28)

In [43]:
df_merged2.shape

(673448, 40)

In [44]:
df_copy.shape

(673448, 16)

In [45]:
#df_.to_hdf('./data/raw/nakon_procesiranja.h5', key='df')

In [46]:
df_merged2.isnull().sum()

KLIJENT_ID                       0
OZNAKA_PARTIJE                   0
DATUM_OTVARANJA                  0
UGOVORENI_IZNOS                  0
VALUTA                           0
VRSTA_KLIJENTA                   0
PROIZVOD                         0
VRSTA_PROIZVODA                  0
TIP_KAMATE                       0
STAROST                          0
PRIJEVREMENI_RASKID              0
NOVI_DATUM_ZATVARANJA2           0
AVG_VISINA_KAMATE                0
PLANIRANA_DULJINA_KREDITA        0
GODINA_OTVARANJA                 0
GODINA_ZATVARANJA                0
ind1-poc                        40
ind2-poc                        40
ind3-poc                        40
ind4-poc                        40
ind5-poc                        40
ind6-poc                        40
ind7-poc                        40
ind8-poc                        40
ind9-poc                        40
ind10-poc                       40
ind11-poc                       40
ind12-poc                       40
ind1-zav            

In [47]:
indikatori.isnull().sum()

General information on Croatia
GDP per capita (in EUR)                                                          0
GDP - year-on-year rate of growth (in %, constant prices)                        0
Average CPI year-on-year inflation rate                                          0
Current account balance (as of % GDP)                                            0
Exports of goods and services (as of % GDP)                                      0
Imports of goods and services (as of % GDP)                                      0
External debt (as of % GDP)                                                      0
Average exchange rate (HRK : 1 EUR)                                              0
Average exchange rate (HRK : 1 USD)                                              0
Consolidated general government net lending (+)/borrowing (-) (million HRK) e    0
Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e    0
General government debt (as % of GDP) e                 

In [48]:
indikatori.head()

General information on Croatia,GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, constant prices)",Average CPI year-on-year inflation rate,Current account balance (as of % GDP),Exports of goods and services (as of % GDP),Imports of goods and services (as of % GDP),External debt (as of % GDP),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+)/borrowing (-) (million HRK) e,Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e,General government debt (as % of GDP) e,"Employment rate (ILO, persons above 15 years of age) f",GODINE
2000,5334.56,3.7675,4.6,-2.18852,35.215,39.6182,51.9406,7.63385,8.28737,....,....,35.4986,42.6,2000
2001,6039.56,3.45241,3.8,-2.91831,37.2483,42.3117,52.3366,7.47101,8.33915,....,....,36.4755,41.8,2001
2002,6632.21,5.25336,1.7,-7.06332,36.24,45.6459,53.036,7.40698,7.87249,-7365.19,-3.48235,36.6133,43.3,2002
2003,7136.85,5.58119,1.8,-5.89579,39.7021,46.3605,64.7067,7.56425,6.70445,-10863.8,-4.67371,38.1037,43.1,2003
2004,7754.34,3.91456,2.1,-4.05455,39.7148,45.6236,68.6055,7.49568,6.03122,-13038.5,-5.20365,40.2784,43.5,2004


In [49]:
s = df_merged2['GODINA_OTVARANJA'].unique()

In [50]:
s.sort()

In [51]:
print(s)

[1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012 2013 2014 2015 2016 2017 2018]


In [52]:
s = df_merged2['GODINA_ZATVARANJA'].unique()
s.sort()
print(s)

[2007 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021
 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035
 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048]


In [53]:
df_merged.head()

,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,...,ind3-poc,ind4-poc,ind5-poc,ind6-poc,ind7-poc,ind8-poc,ind9-poc,ind10-poc,ind11-poc,ind12-poc
0,772139,5521579,2010-06-11,46000.00,1,1410,FL0801,A,B,38,...,1.1,-1.0676,37.6476,37.982,103.841,7.28623,5.50002,-6.27526,57.3479,46.5
1,481985,7443823,2013-01-23,44679.50,2,1550,FD0100,L,A,37,...,2.2,0.945846,42.8399,42.4593,104.554,7.57355,5.70588,-5.2795,80.3993,42.075
2,139014,6511639,2011-09-16,29959.15,2,1410,FL0801,A,B,49,...,2.3,-0.698502,40.415,40.8333,103.44,7.4342,5.34351,-7.86303,63.8686,44.8
3,952566,6143123,2011-05-23,100000.00,1,1410,FL0801,A,B,35,...,2.3,-0.698502,40.415,40.8333,103.44,7.4342,5.34351,-7.86303,63.8686,44.8
4,686960,7791434,2013-05-27,4017.00,1,1410,FL1100,A,A,65,...,2.2,0.945846,42.8399,42.4593,104.554,7.57355,5.70588,-5.2795,80.3993,42.075


In [54]:
df_merged['PRIJEVREMENI_RASKID']

0         N
1         N
2         N
3         N
4         N
5         N
6         N
7         N
8         Y
9         Y
10        Y
11        N
12        Y
13        N
14        Y
15        N
16        Y
17        N
18        Y
19        Y
20        N
21        N
22        N
23        N
24        N
25        N
26        N
27        N
28        N
29        N
         ..
673418    N
673419    N
673420    N
673421    N
673422    N
673423    N
673424    N
673425    N
673426    N
673427    N
673428    N
673429    N
673430    N
673431    N
673432    N
673433    N
673434    N
673435    N
673436    N
673437    N
673438    N
673439    N
673440    N
673441    N
673442    N
673443    N
673444    N
673445    N
673446    N
673447    N
Name: PRIJEVREMENI_RASKID, Length: 673448, dtype: object

In [55]:
df_merged.shape

(673448, 28)

In [56]:
df_merged.columns
#moram na test setu dodat godinu otvaranja

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'UGOVORENI_IZNOS',
       'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA2',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'GODINA_ZATVARANJA', 'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc',
       'ind5-poc', 'ind6-poc', 'ind7-poc', 'ind8-poc', 'ind9-poc', 'ind10-poc',
       'ind11-poc', 'ind12-poc'],
      dtype='object')

In [57]:
del_col_list = ['NOVI_DATUM_ZATVARANJA2','GODINA_ZATVARANJA','DATUM_OTVARANJA']

In [58]:
df_merged.drop(del_col_list,axis=1,inplace=True)

In [59]:
df_merged.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'AVG_VISINA_KAMATE',
       'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA', 'ind1-poc', 'ind2-poc',
       'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc', 'ind7-poc', 'ind8-poc',
       'ind9-poc', 'ind10-poc', 'ind11-poc', 'ind12-poc'],
      dtype='object')

In [60]:
mydateparser = lambda c: pd.to_datetime(c, format='%d.%m.%Y', errors='coerce')

test = pd.read_excel('C:/Users/e_len/Documents/eval_dataset_nan.xlsx')#parse_dates=['DATUM_OTVARANJA', 'PLANIRANI_DATUM_ZATVARANJA'], date_parser=mydateparser

In [61]:
test['DATUM_OTVARANJA'] = pd.to_datetime(test['DATUM_OTVARANJA'], format='%d.%m.%Y')

In [63]:
test

,instance_id,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID
0,0,1049000,5585453,2012-07-14,2013-07-15,273555.01,2,1410,TM0109,L,3.60,A,64,NaN
1,1,481302,752335,2016-01-24,2016-01-24,50970.78,2,1410,TM0109,L,0.20,A,56,NaN
2,2,838192,6623548,2018-12-26,2018-12-26,8969.36,2,1410,TM0109,L,0.02,A,52,NaN
3,3,174575,425060,2012-08-04,2012-08-04,16142.48,2,1410,TM0109,L,2.30,C,76,NaN
4,4,14884,1183149,2011-03-27,2012-03-28,45315.31,2,1410,TM0109,L,3.30,C,53,NaN
5,5,1348702,12847410,2018-08-04,2020-02-15,40000.00,1,1410,TM0109,L,1.50,A,35,NaN
6,6,764915,5647773,2013-12-21,2014-03-22,119043.94,2,1410,TM0109,L,2.00,A,50,NaN
7,7,367476,4875789,2010-01-08,2011-01-09,103650.71,2,1410,TM0109,L,5.15,A,35,NaN
8,8,234350,12636299,2017-08-08,2018-08-09,185000.00,1,1550,TM0109,L,1.30,A,46,NaN
9,9,888283,9088014,2018-11-23,2018-11-23,54123.73,1,1410,TM0109,L,0.20,A,38,NaN


In [64]:
test.columns

Index(['instance_id', 'KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA',
       'PLANIRANI_DATUM_ZATVARANJA', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'VISINA_KAMATE',
       'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID'],
      dtype='object')

In [65]:
test

,instance_id,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID
0,0,1049000,5585453,2012-07-14,2013-07-15,273555.01,2,1410,TM0109,L,3.60,A,64,NaN
1,1,481302,752335,2016-01-24,2016-01-24,50970.78,2,1410,TM0109,L,0.20,A,56,NaN
2,2,838192,6623548,2018-12-26,2018-12-26,8969.36,2,1410,TM0109,L,0.02,A,52,NaN
3,3,174575,425060,2012-08-04,2012-08-04,16142.48,2,1410,TM0109,L,2.30,C,76,NaN
4,4,14884,1183149,2011-03-27,2012-03-28,45315.31,2,1410,TM0109,L,3.30,C,53,NaN
5,5,1348702,12847410,2018-08-04,2020-02-15,40000.00,1,1410,TM0109,L,1.50,A,35,NaN
6,6,764915,5647773,2013-12-21,2014-03-22,119043.94,2,1410,TM0109,L,2.00,A,50,NaN
7,7,367476,4875789,2010-01-08,2011-01-09,103650.71,2,1410,TM0109,L,5.15,A,35,NaN
8,8,234350,12636299,2017-08-08,2018-08-09,185000.00,1,1550,TM0109,L,1.30,A,46,NaN
9,9,888283,9088014,2018-11-23,2018-11-23,54123.73,1,1410,TM0109,L,0.20,A,38,NaN


In [66]:
del test['instance_id']

In [67]:
test.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA',
       'PLANIRANI_DATUM_ZATVARANJA', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'VISINA_KAMATE',
       'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID'],
      dtype='object')

In [68]:
df_merged.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'AVG_VISINA_KAMATE',
       'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA', 'ind1-poc', 'ind2-poc',
       'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc', 'ind7-poc', 'ind8-poc',
       'ind9-poc', 'ind10-poc', 'ind11-poc', 'ind12-poc'],
      dtype='object')

In [69]:
del test['OZNAKA_PARTIJE']

In [70]:
del df_merged['OZNAKA_PARTIJE'] # zapravo radi, ali sam vec prije obrisala

In [71]:
test['GODINA_OTVARANJA'] = test['DATUM_OTVARANJA'].dt.year

In [72]:
test['PLANIRANI_DATUM_ZATVARANJA']

0       2013-07-15
1       2016-01-24
2       2018-12-26
3       2012-08-04
4       2012-03-28
5       2020-02-15
6       2014-03-22
7       2011-01-09
8       2018-08-09
9       2018-11-23
10      2015-06-17
11      2014-11-21
12      2011-04-02
13      2013-01-23
14      2018-12-05
15      2016-04-19
16      2015-10-16
17      2016-06-30
18      2011-06-08
19      2017-03-23
20      2017-07-19
21      2018-10-13
22      2013-04-10
23      2011-04-30
24      2013-08-16
25      2012-05-15
26      2011-02-09
27      2017-06-25
28      2012-11-19
29      2013-04-21
           ...    
84873   2017-03-31
84874   2013-12-31
84875   2014-03-31
84876   2014-06-30
84877   2013-12-31
84878   2017-03-31
84879   2013-12-31
84880   2013-12-31
84881   2013-09-30
84882   2014-12-31
84883   2016-03-31
84884   2014-09-30
84885   2014-03-28
84886   2026-12-30
84887   2014-06-02
84888   2014-12-31
84889   2026-02-02
84890   2016-07-28
84891   2013-12-19
84892   2011-06-28
84893   2018-07-04
84894   2021

In [73]:
test['PLANIRANA_DULJINA_KREDITA'] = (test['PLANIRANI_DATUM_ZATVARANJA']-test['DATUM_OTVARANJA']).dt.days

In [74]:
test['PLANIRANA_DULJINA_KREDITA']

0         366
1           0
2           0
3           0
4         367
5         560
6          91
7         366
8         366
9           0
10         93
11        366
12       2364
13        367
14          0
15          0
16        178
17        184
18       1096
19         91
20        366
21        -63
22        183
23         61
24        182
25        465
26       1652
27        732
28        185
29          0
         ... 
84873    2019
84874    1183
84875    1187
84876    1217
84877    1183
84878    1564
84879    1183
84880    1183
84881    1223
84882     737
84883    1136
84884    1160
84885     820
84886    5479
84887    1162
84888    3778
84889    3715
84890    3838
84891    2555
84892    1880
84893    1462
84894    2194
84895     367
84896       7
84897       4
84898       7
84899    1826
84900     304
84901    1826
84902    1826
Name: PLANIRANA_DULJINA_KREDITA, Length: 84903, dtype: int64

In [75]:
del test['DATUM_OTVARANJA']

In [76]:

df_merged.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc',
       'ind7-poc', 'ind8-poc', 'ind9-poc', 'ind10-poc', 'ind11-poc',
       'ind12-poc'],
      dtype='object')

In [77]:
test.columns

Index(['KLIJENT_ID', 'PLANIRANI_DATUM_ZATVARANJA', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'VISINA_KAMATE',
       'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID', 'GODINA_OTVARANJA',
       'PLANIRANA_DULJINA_KREDITA'],
      dtype='object')

In [78]:
del test['PLANIRANI_DATUM_ZATVARANJA']

In [79]:
test['AVG_VISINA_KAMATE'] = test['VISINA_KAMATE']

In [80]:
test.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'VISINA_KAMATE', 'TIP_KAMATE', 'STAROST',
       'PRIJEVREMENI_RASKID', 'GODINA_OTVARANJA', 'PLANIRANA_DULJINA_KREDITA',
       'AVG_VISINA_KAMATE'],
      dtype='object')

In [81]:
del test['VISINA_KAMATE']

In [82]:
test.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID',
       'GODINA_OTVARANJA', 'PLANIRANA_DULJINA_KREDITA', 'AVG_VISINA_KAMATE'],
      dtype='object')

In [83]:
df_merged.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc',
       'ind7-poc', 'ind8-poc', 'ind9-poc', 'ind10-poc', 'ind11-poc',
       'ind12-poc'],
      dtype='object')

In [ ]:
test = test[['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA']]

In [88]:
test

,KLIJENT_ID,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA,GODINA_OTVARANJA
0,1049000,273555.01,2,1410,TM0109,L,A,64,NaN,3.60,366,2012
1,481302,50970.78,2,1410,TM0109,L,A,56,NaN,0.20,0,2016
2,838192,8969.36,2,1410,TM0109,L,A,52,NaN,0.02,0,2018
3,174575,16142.48,2,1410,TM0109,L,C,76,NaN,2.30,0,2012
4,14884,45315.31,2,1410,TM0109,L,C,53,NaN,3.30,367,2011
5,1348702,40000.00,1,1410,TM0109,L,A,35,NaN,1.50,560,2018
6,764915,119043.94,2,1410,TM0109,L,A,50,NaN,2.00,91,2013
7,367476,103650.71,2,1410,TM0109,L,A,35,NaN,5.15,366,2010
8,234350,185000.00,1,1550,TM0109,L,A,46,NaN,1.30,366,2017
9,888283,54123.73,1,1410,TM0109,L,A,38,NaN,0.20,0,2018


In [89]:
test.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA'],
      dtype='object')

In [90]:
test_merged = pd.merge(test,indikatori,left_on=['GODINA_OTVARANJA'], right_on=['GODINE'], how='left')
test_merged.rename(columns={'GDP per capita (in EUR)  ': 'ind1-poc', 'GDP - year-on-year rate of growth (in %, constant prices) ': 'ind2-poc', 
                   'Average CPI year-on-year inflation rate ': 'ind3-poc', 'Current account balance (as of % GDP) ': 'ind4-poc', 
                   'Exports of goods and services (as of % GDP) ': 'ind5-poc', 'Imports of goods and services (as of % GDP) ': 'ind6-poc',
                  'External debt (as of % GDP) ': 'ind7-poc', 'Average exchange rate (HRK : 1 EUR) ': 'ind8-poc', 'Average exchange rate (HRK : 1 USD) ': 'ind9-poc',
                  'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e' : 'ind10-poc', 'General government debt (as % of GDP) e': 'ind11-poc',
                  'Employment rate (ILO, persons above 15 years of age) f': 'ind12-poc'}, inplace=True)

In [91]:
test_merged.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc',
       'ind7-poc', 'ind8-poc', 'ind9-poc',
       'Consolidated general government net lending (+)/borrowing (-) (million HRK) e',
       'ind10-poc', 'ind11-poc', 'ind12-poc', 'GODINE'],
      dtype='object')

In [92]:
del test_merged['Consolidated general government net lending (+)/borrowing (-) (million HRK) e']

In [93]:
test_merged.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc',
       'ind7-poc', 'ind8-poc', 'ind9-poc', 'ind10-poc', 'ind11-poc',
       'ind12-poc', 'GODINE'],
      dtype='object')

In [94]:
test_merged

,KLIJENT_ID,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,AVG_VISINA_KAMATE,...,ind4-poc,ind5-poc,ind6-poc,ind7-poc,ind8-poc,ind9-poc,ind10-poc,ind11-poc,ind12-poc,GODINE
0,1049000,273555.01,2,1410,TM0109,L,A,64,NaN,3.60,...,-0.114238,41.6252,41.1878,102.929,7.51734,5.85086,-5.2655,69.4446,43.2,2012.0
1,481302,50970.78,2,1410,TM0109,L,A,56,NaN,0.20,...,2.58553,48.8269,45.9917,89.2949,7.52938,6.80372,-0.932065,80.1886,44.5847,2016.0
2,838192,8969.36,2,1410,TM0109,L,A,52,NaN,0.02,...,-,-,-,75.0706,7.41411,6.27841,-,-,-,2018.0
3,174575,16142.48,2,1410,TM0109,L,C,76,NaN,2.30,...,-0.114238,41.6252,41.1878,102.929,7.51734,5.85086,-5.2655,69.4446,43.2,2012.0
4,14884,45315.31,2,1410,TM0109,L,C,53,NaN,3.30,...,-0.698502,40.415,40.8333,103.44,7.4342,5.34351,-7.86303,63.8248,44.8,2011.0
5,1348702,40000.00,1,1410,TM0109,L,A,35,NaN,1.50,...,-,-,-,75.0706,7.41411,6.27841,-,-,-,2018.0
6,764915,119043.94,2,1410,TM0109,L,A,50,NaN,2.00,...,0.946094,42.8402,42.4593,104.554,7.57355,5.70588,-5.2795,80.3602,42.075,2013.0
7,367476,103650.71,2,1410,TM0109,L,A,35,NaN,5.15,...,-1.0676,37.6476,37.982,103.841,7.28623,5.50002,-6.27526,57.3055,46.5,2010.0
8,234350,185000.00,1,1550,TM0109,L,A,46,NaN,1.30,...,4.04705,51.27,49.1009,81.7513,7.4601,6.6224,0.863547,77.4848,45.8246,2017.0
9,888283,54123.73,1,1410,TM0109,L,A,38,NaN,0.20,...,-,-,-,75.0706,7.41411,6.27841,-,-,-,2018.0


In [95]:
df_merged.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc',
       'ind7-poc', 'ind8-poc', 'ind9-poc', 'ind10-poc', 'ind11-poc',
       'ind12-poc'],
      dtype='object')

In [96]:
del test_merged['GODINE']

In [97]:
X_train = df_merged[['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc',
       'ind7-poc', 'ind8-poc', 'ind9-poc', 'ind10-poc', 'ind11-poc',
       'ind12-poc']]

In [98]:
X_test = test_merged[['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'TIP_KAMATE', 'STAROST',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc',
       'ind7-poc', 'ind8-poc', 'ind9-poc', 'ind10-poc', 'ind11-poc',
       'ind12-poc']]

In [99]:
X_test

,KLIJENT_ID,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA,...,ind3-poc,ind4-poc,ind5-poc,ind6-poc,ind7-poc,ind8-poc,ind9-poc,ind10-poc,ind11-poc,ind12-poc
0,1049000,273555.01,2,1410,TM0109,L,A,64,3.60,366,...,3.4,-0.114238,41.6252,41.1878,102.929,7.51734,5.85086,-5.2655,69.4446,43.2
1,481302,50970.78,2,1410,TM0109,L,A,56,0.20,0,...,-1.1,2.58553,48.8269,45.9917,89.2949,7.52938,6.80372,-0.932065,80.1886,44.5847
2,838192,8969.36,2,1410,TM0109,L,A,52,0.02,0,...,1.5,-,-,-,75.0706,7.41411,6.27841,-,-,-
3,174575,16142.48,2,1410,TM0109,L,C,76,2.30,0,...,3.4,-0.114238,41.6252,41.1878,102.929,7.51734,5.85086,-5.2655,69.4446,43.2
4,14884,45315.31,2,1410,TM0109,L,C,53,3.30,367,...,2.3,-0.698502,40.415,40.8333,103.44,7.4342,5.34351,-7.86303,63.8248,44.8
5,1348702,40000.00,1,1410,TM0109,L,A,35,1.50,560,...,1.5,-,-,-,75.0706,7.41411,6.27841,-,-,-
6,764915,119043.94,2,1410,TM0109,L,A,50,2.00,91,...,2.2,0.946094,42.8402,42.4593,104.554,7.57355,5.70588,-5.2795,80.3602,42.075
7,367476,103650.71,2,1410,TM0109,L,A,35,5.15,366,...,1.1,-1.0676,37.6476,37.982,103.841,7.28623,5.50002,-6.27526,57.3055,46.5
8,234350,185000.00,1,1550,TM0109,L,A,46,1.30,366,...,1.1,4.04705,51.27,49.1009,81.7513,7.4601,6.6224,0.863547,77.4848,45.8246
9,888283,54123.73,1,1410,TM0109,L,A,38,0.20,0,...,1.5,-,-,-,75.0706,7.41411,6.27841,-,-,-


In [100]:
y_train = df_merged['PRIJEVREMENI_RASKID']

In [101]:
y_test = test_merged['PRIJEVREMENI_RASKID']

In [102]:
X_train['train']=1
X_test['train']=0

In [103]:
combined = pd.concat([X_train,X_test])

In [104]:
dumm = pd.get_dummies(combined['VRSTA_PROIZVODA'])

In [105]:
combined = pd.concat([combined,dumm],axis = 1)

In [106]:
combined.head()

,KLIJENT_ID,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA,...,ind6-poc,ind7-poc,ind8-poc,ind9-poc,ind10-poc,ind11-poc,ind12-poc,train,A,L
0,772139,46000.00,1,1410,FL0801,A,B,38,9.70,2576,...,37.982,103.841,7.28623,5.50002,-6.27526,57.3055,46.5,1,1,0
1,481985,44679.50,2,1550,FD0100,L,A,37,3.10,1862,...,42.4593,104.554,7.57355,5.70588,-5.2795,80.3602,42.075,1,0,1
2,139014,29959.15,2,1410,FL0801,A,B,49,7.95,1841,...,40.8333,103.44,7.4342,5.34351,-7.86303,63.8248,44.8,1,1,0
3,952566,100000.00,1,1410,FL0801,A,B,35,9.70,1104,...,40.8333,103.44,7.4342,5.34351,-7.86303,63.8248,44.8,1,1,0
4,686960,4017.00,1,1410,FL1100,A,A,65,11.95,734,...,42.4593,104.554,7.57355,5.70588,-5.2795,80.3602,42.075,1,1,0


In [107]:
dumm = pd.get_dummies(combined['VALUTA'])

In [108]:
combined = pd.concat([combined,dumm],axis = 1)

In [109]:
combined.head()

,KLIJENT_ID,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA,...,ind11-poc,ind12-poc,train,A,L,1,2,3,4,5
0,772139,46000.00,1,1410,FL0801,A,B,38,9.70,2576,...,57.3055,46.5,1,1,0,1,0,0,0,0
1,481985,44679.50,2,1550,FD0100,L,A,37,3.10,1862,...,80.3602,42.075,1,0,1,0,1,0,0,0
2,139014,29959.15,2,1410,FL0801,A,B,49,7.95,1841,...,63.8248,44.8,1,1,0,0,1,0,0,0
3,952566,100000.00,1,1410,FL0801,A,B,35,9.70,1104,...,63.8248,44.8,1,1,0,1,0,0,0,0
4,686960,4017.00,1,1410,FL1100,A,A,65,11.95,734,...,80.3602,42.075,1,1,0,1,0,0,0,0


In [110]:
dumm = pd.get_dummies(combined['TIP_KAMATE'])

In [111]:
combined = pd.concat([combined,dumm],axis = 1)

In [112]:
combined.head()

,KLIJENT_ID,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA,...,L,1,2,3,4,5,A,B,C,D
0,772139,46000.00,1,1410,FL0801,A,B,38,9.70,2576,...,0,1,0,0,0,0,0,1,0,0
1,481985,44679.50,2,1550,FD0100,L,A,37,3.10,1862,...,1,0,1,0,0,0,1,0,0,0
2,139014,29959.15,2,1410,FL0801,A,B,49,7.95,1841,...,0,0,1,0,0,0,0,1,0,0
3,952566,100000.00,1,1410,FL0801,A,B,35,9.70,1104,...,0,1,0,0,0,0,0,1,0,0
4,686960,4017.00,1,1410,FL1100,A,A,65,11.95,734,...,0,1,0,0,0,0,1,0,0,0


In [113]:
del combined['VALUTA'];del combined['VRSTA_PROIZVODA'];del combined['TIP_KAMATE'];

In [114]:
combined.head()

,KLIJENT_ID,UGOVORENI_IZNOS,VRSTA_KLIJENTA,PROIZVOD,STAROST,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA,GODINA_OTVARANJA,ind1-poc,ind2-poc,...,L,1,2,3,4,5,A,B,C,D
0,772139,46000.00,1410,FL0801,38,9.70,2576,2010,10530.3,-1.47292,...,0,1,0,0,0,0,0,1,0,0
1,481985,44679.50,1550,FD0100,37,3.10,1862,2013,10293.3,-0.491749,...,1,0,1,0,0,0,1,0,0,0
2,139014,29959.15,1410,FL0801,49,7.95,1841,2011,10480,-0.34019,...,0,0,1,0,0,0,0,1,0,0
3,952566,100000.00,1410,FL0801,35,9.70,1104,2011,10480,-0.34019,...,0,1,0,0,0,0,0,1,0,0
4,686960,4017.00,1410,FL1100,65,11.95,734,2013,10293.3,-0.491749,...,0,1,0,0,0,0,1,0,0,0


In [115]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
combined['VRSTA_KLIJENTA_CODE'] = lb_make.fit_transform(combined['VRSTA_KLIJENTA'])

In [116]:
combined['PROIZVOD_CODE'] = lb_make.fit_transform(combined['PROIZVOD'])

In [117]:
combined['GODINA_OTVARANJA_CODE'] = lb_make.fit_transform(combined['GODINA_OTVARANJA'])

In [118]:
del combined['GODINA_OTVARANJA'];del combined['VRSTA_KLIJENTA'];del combined['PROIZVOD'];

In [119]:
combined.head()

,KLIJENT_ID,UGOVORENI_IZNOS,STAROST,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA,ind1-poc,ind2-poc,ind3-poc,ind4-poc,ind5-poc,...,3,4,5,A,B,C,D,VRSTA_KLIJENTA_CODE,PROIZVOD_CODE,GODINA_OTVARANJA_CODE
0,772139,46000.00,38,9.70,2576,10530.3,-1.47292,1.1,-1.0676,37.6476,...,0,0,0,0,1,0,0,7,6,14
1,481985,44679.50,37,3.10,1862,10293.3,-0.491749,2.2,0.946094,42.8402,...,0,0,0,1,0,0,0,11,0,17
2,139014,29959.15,49,7.95,1841,10480,-0.34019,2.3,-0.698502,40.415,...,0,0,0,0,1,0,0,7,6,15
3,952566,100000.00,35,9.70,1104,10480,-0.34019,2.3,-0.698502,40.415,...,0,0,0,0,1,0,0,7,6,15
4,686960,4017.00,65,11.95,734,10293.3,-0.491749,2.2,0.946094,42.8402,...,0,0,0,1,0,0,0,7,12,17


In [120]:
combined['ind1-poc'] = combined['ind1-poc'].astype(float)
combined['ind2-poc'] = combined['ind2-poc'].astype(float)
combined['ind3-poc'] = combined['ind3-poc'].astype(float)


In [121]:
#combined['ind4-poc'] = combined['ind4-poc'].astype(float)
#combined['ind5-poc'] = combined['ind5-poc'].astype(float)
#combined['ind6-poc'] = combined['ind6-poc'].astype(float)

In [122]:
combined.loc[combined['ind4-poc'] == '-','ind4-poc'] = np.nan
combined.loc[combined['ind5-poc'] == '-','ind5-poc'] = np.nan
combined.loc[combined['ind6-poc'] == '-','ind6-poc'] = np.nan
combined.loc[combined['ind10-poc'] == '-','ind10-poc'] = np.nan
combined.loc[combined['ind11-poc'] == '-','ind11-poc'] = np.nan
combined.loc[combined['ind12-poc'] == '-','ind12-poc'] = np.nan

In [123]:
combined['ind4-poc'] = combined['ind4-poc'].astype(float)
combined['ind5-poc'] = combined['ind5-poc'].astype(float)
combined['ind6-poc'] = combined['ind6-poc'].astype(float)
#combined['ind10-poc'] = combined['ind10-poc'].astype(float)
combined['ind11-poc'] = combined['ind11-poc'].astype(float)
combined['ind12-poc'] = combined['ind12-poc'].astype(float)

In [124]:
combined.loc[combined['ind10-poc'] == '....','ind10-poc'] = np.nan
combined['ind10-poc'] = combined['ind10-poc'].astype(float)

In [125]:
combined['ind7-poc'] = combined['ind7-poc'].astype(float)
combined['ind8-poc'] = combined['ind8-poc'].astype(float)
combined['ind9-poc'] = combined['ind9-poc'].astype(float)

In [126]:
y_train = lb_make.fit_transform(y_train)

In [149]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [127]:
#y_test = lb_make.fit_transform(y_test)

In [128]:
X_train = combined[combined['train']==1]
X_test = combined[combined['train']==0]

In [129]:
X_train.drop(['train'],axis=1,inplace=True)
X_test.drop(['train'],axis=1,inplace=True)

C:\Users\e_len\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [130]:
X_train.head()

,KLIJENT_ID,UGOVORENI_IZNOS,STAROST,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA,ind1-poc,ind2-poc,ind3-poc,ind4-poc,ind5-poc,...,3,4,5,A,B,C,D,VRSTA_KLIJENTA_CODE,PROIZVOD_CODE,GODINA_OTVARANJA_CODE
0,772139,46000.00,38,9.70,2576,10530.260615,-1.472916,1.1,-1.067602,37.647589,...,0,0,0,0,1,0,0,7,6,14
1,481985,44679.50,37,3.10,1862,10293.323129,-0.491749,2.2,0.946094,42.840161,...,0,0,0,1,0,0,0,11,0,17
2,139014,29959.15,49,7.95,1841,10479.994090,-0.340190,2.3,-0.698502,40.414985,...,0,0,0,0,1,0,0,7,6,15
3,952566,100000.00,35,9.70,1104,10479.994090,-0.340190,2.3,-0.698502,40.414985,...,0,0,0,0,1,0,0,7,6,15
4,686960,4017.00,65,11.95,734,10293.323129,-0.491749,2.2,0.946094,42.840161,...,0,0,0,1,0,0,0,7,12,17


In [131]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 673448 entries, 0 to 673447
Data columns (total 31 columns):
KLIJENT_ID                   673448 non-null int64
UGOVORENI_IZNOS              673448 non-null float64
STAROST                      673448 non-null int64
AVG_VISINA_KAMATE            673448 non-null float64
PLANIRANA_DULJINA_KREDITA    673448 non-null int64
ind1-poc                     673408 non-null float64
ind2-poc                     673408 non-null float64
ind3-poc                     673408 non-null float64
ind4-poc                     637170 non-null float64
ind5-poc                     637170 non-null float64
ind6-poc                     637170 non-null float64
ind7-poc                     673408 non-null float64
ind8-poc                     673408 non-null float64
ind9-poc                     673408 non-null float64
ind10-poc                    634689 non-null float64
ind11-poc                    637170 non-null float64
ind12-poc                    637170 non-null fl

In [132]:
from sklearn.preprocessing import Imputer

In [133]:
# Create an imputer object that looks for 'Nan' values, then replaces them with the mean value of the feature by columns (axis=0)
mean_imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)

# Train the imputor on the df dataset
imputed_df = pd.DataFrame(mean_imputer.fit_transform(X_train))
imputed_df.columns = X_train.columns
imputed_df.index = X_train.index

C:\Users\e_len\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [140]:
X_train = imputed_df

In [135]:
# Train the imputor on the df dataset
imputed_df1 = pd.DataFrame(mean_imputer.fit_transform(X_test))
imputed_df1.columns = X_test.columns
imputed_df1.index = X_test.index

In [138]:
X_test=imputed_df1

In [141]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=12)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [145]:
np.unique(y_pred)

array([0, 1])

In [146]:
#Import scikit-learn metrics module for accuracy calculation
#from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred)) --> nemam test podatke

In [147]:
y_test = y_pred

In [148]:
student = pd.read_excel('C:/Users/e_len/Documents/eval_dataset_nan.xlsx')

In [155]:
y_test = np.where(y_test==0,'N', 'Y')

In [156]:
y_test

array(['N', 'N', 'N', ..., 'N', 'N', 'N'], dtype='<U1')

In [157]:
student['PRIJEVREMENI_RASKID'] = y_test

In [159]:
student['PRIJEVREMENI_RASKID'].unique()

array(['N', 'Y'], dtype=object)

In [162]:
student.to_csv('C:/Users/e_len/Documents/student.csv')

In [164]:
from sklearn import preprocessing

x = X_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
new_X_train = pd.DataFrame(x_scaled)

In [165]:
new_X_train.columns = X_train.columns
new_X_train.index = X_train.index

In [166]:
X_train = new_X_train

In [167]:
x = X_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
new_X_test = pd.DataFrame(x_scaled)

In [168]:
new_X_test.columns = X_test.columns
new_X_test.index = X_test.index

In [169]:
X_test = new_X_test

In [170]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=50)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [171]:
y_test = y_pred
y_test = np.where(y_test==0,'N', 'Y')
student['PRIJEVREMENI_RASKID'] = y_test
student.to_csv('C:/Users/e_len/Documents/student.csv')